In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print('length of dataset in characters: ', len(text))

length of dataset in characters:  1115394


In [3]:
# look at first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# find all the unique characters that occur
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# tokenize the input text - raw text to sequence of integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s : [stoi[c] for c in s] # encoder takes string and turns into list of integers
decode = lambda l : ''.join([itos[i] for i in l]) # decoder takes list of integers and turns into string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long) # list of integers into torch tensor
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# split into training and validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
# don't give anything all the characters at once
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
# How big are our chunks of text?
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1] # beginning all the way up to t
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [10]:
# How many chunks in each batch tensor?
torch.manual_seed(1337)
batch_size = 4 # number of independent sequences to process in parallel
block_size = 8 # maximum size 'context' for each prediction

def get_batch(split: str) -> tuple[torch.tensor]:
    """Generate a small batch of data inputs x and targets y

    Args:
        split (str): specification on if this will be a training batch

    Returns:
        tuple[torch.tensor]: inputs and targets
    """
    data = train_data if split == 'train' else val_data
    # indices must be lower than len(data) - block_size so each batch can achieve the whole context
    indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in indices]) # 2D array-esque - a tensor containing tensors to represent a batch of inputs
    y = torch.stack([data[i+1:i+block_size+1] for i in indices])
    return x, y
    
x_batch, y_batch = get_batch('train')
print('inputs:')
print(x_batch.shape)
print(x_batch)
print('targets:')
print(y_batch.shape)
print(y_batch)

print('---------------------')

for input_idx in range(batch_size): # batch dimension
    for element_index in range(block_size): # context dimension
        context = x_batch[input_idx, :element_index+1] # the context for the given element of the batch
        target = y_batch[input_idx, element_index] # only the preceding following character
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------------------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when

In [11]:
print(x_batch)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    # ultimately two characters together can affect which character will precede - so we want to be able to tell what each character paired with any other character "means"
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # thin wrapper around a tensor of size (vocab_size,vocab_size)
        
    def forward(self, batch_input, targets=None):
        
        # batch_input and targets are both (batch_size, context_size) tensor of integers
        logits = self.token_embedding_table(batch_input) # (batch_size,context_size,vocab_size)
        
        if targets is None:
            loss = None
        else:
            batches, num_characters, alphabet_size = logits.shape
            # stretch out each batch to one list
            logits = logits.view(batches*num_characters, alphabet_size)
            targets = targets.view(batches*num_characters)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, batch_input, max_new_tokens):
        # this is overkill for a bigram model because only the previous character is used for the next prediction even though we give it the whole model... but it's scalable
        # again, batch_input is a (batch_size,input_size) tensor
        for _ in range(max_new_tokens):
            # get predictions
            logits, _ = self(batch_input) # forward function
            logits = logits[:, -1, :] # get your hands on the LAST character of each context - that will predict our next character - dimension (batch_size, vocab_size)
            probs = F.softmax(logits, dim=-1) # for each last character in our batch, generate a list of probabilities for the next character to precede it
            next_characters = torch.multinomial(probs, num_samples=1)
            batch_input = torch.cat((batch_input, next_characters), dim=1) # (batch_size, input_size + 1)
        return batch_input
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(x_batch, y_batch)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [13]:
batch_of_strings = torch.zeros((1,1), dtype=torch.long) # kick off the generation with a batch of size 1 and input size of 1 character
print(decode(m.generate(batch_of_strings, max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [14]:
# create an optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    input_batch, output_batch = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(input_batch, output_batch)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item()) # numerical value only - not the tensore we just took the gradient of

2.382369041442871


In [16]:
print(decode(m.generate(batch_of_strings, max_new_tokens=300)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecor


### Mathematical trick in self-attention...

In [17]:
# Consider the following example:

torch.manual_seed(1337)
batch_size, characters_per_time_slice, alphabet_size = 4,8,2 # (batch, time, channels (B,T,C))
x = torch.randn(batch_size, characters_per_time_slice, alphabet_size)
x.shape

torch.Size([4, 8, 2])

In [18]:
# We want x[b,t] = mean_{i<=t} x[b,i] - take the average of all preceding tokens
xbow = torch.zeros((batch_size, characters_per_time_slice, alphabet_size))
# (bow - bag of words - since we have a word stored on each of the locations)
for b_idx in range(batch_size):
    for t_idx in range(characters_per_time_slice):
        xprev = x[b_idx,:t_idx+1] # (t_idx,C) - each included character still has all its channels
        xbow[b_idx,t_idx] = torch.mean(xprev, 0)

In [19]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [20]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [21]:
# We can be faster calculating running averages with matrix multiplication...
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3)) # lower triangular
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [22]:
wei = torch.tril(torch.ones(characters_per_time_slice, characters_per_time_slice))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [23]:
# Batch matrix multiply DOES work to efficiently calculate our aggregate sums and averages!
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [24]:
# 3rd version... using softmax - which allows different characters to pay more attention to each other
tril = torch.tril(torch.ones(characters_per_time_slice, characters_per_time_slice)) # lower triangular ones
wei = torch.zeros((characters_per_time_slice, characters_per_time_slice))
# for all elements where til is zero, make that -inf for wei
wei = wei.masked_fill(tril == 0, float('-inf'))
# softmax along each row is a normalization optimization -> 0 becomes 1 and -inf becomes 0
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [42]:
# version 4: self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch size, number of time steps, alphabet size
x = torch.randn(B,T,C)

# each token has two vectors - a key and a query - and it goes around dot-producting its query with other tokens' keys - this is how tokens interact
head_size = 16 # head size is a hyperparameter
key = nn.Linear(C, head_size, bias=False) # a vector for each character in the alphabet (C, head_size)
query = nn.Linear(C, head_size, bias=False) # same
value = nn.Linear(C, head_size, bias=False) # same
# now we want every token in each batch to have a vector of keys and a vector of queries
k = key(x) # (B,T,head_size), because (B,T,C) @ (C,head_size) -> (B,T,head_size)
q = query(x) # (B,T,head_size) 
# matrix multiplication being done in parallel across the batch dimension
wei = q @ k.transpose(-2,-1) # transpose the last two dimensions of k so that (B, T, head_size) @ (B, head_size, T) -> (B, T, T)
# now for each batch, each character/token in the batch has a real-valued affinity with all other characters/tokens in said batch
# HOWEVER, we want an attention 'normalizer' so that variance can be 1
wei = wei * head_size ** -0.5

# NOW wei will be data-dependent
tril = torch.tril(torch.ones(T, T)) # upper triangular matrix
wei = wei.masked_fill(tril == 0, float('-inf')) # prevent nodes from future talking to nodes from the past
wei = F.softmax(wei, dim=-1)

v = value(x) # v is vector we aggregate instead of raw x
out = wei @ v # (B, T, head_size)

out.shape

torch.Size([4, 8, 16])

In [43]:
q.var()

tensor(0.3386, grad_fn=<VarBackward0>)

In [45]:
k.var()

tensor(0.3164, grad_fn=<VarBackward0>)

In [44]:
wei.var()

tensor(0.0287, grad_fn=<VarBackward0>)

In [46]:
wei[0] # how much information to aggregate from any of the tokens in the past?

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward0>)

In [ ]:
class BatchNorm1d:
    pass